In [121]:
import cv2
import itertools as it
import matplotlib.pyplot as plt
import numpy as np


MAZE_FILE_NAME = '../Maze.png'
ROBOT_FILE_NAME = '../Robot.png'
IMAGE_LADYBUG_FILE_NAME = '../Ladybug_small.png'
MAP_FILE_NAME = '../MapBuilt.txt'

def readImage():
    '''
    Reads the maze image and shows it.
    '''
    image = cv2.imread(MAZE_FILE_NAME, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image


def maskColour(image, lower, upper):
    '''
    Given a lower and upper bound for a HSV colour, returns a binary image that is white-focused.
    '''
    temp_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    mask = cv2.inRange(temp_image, lower, upper)
    kernel = np.ones((9, 9), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    return mask


def markFeature(image, mask, colour):
    '''
    Mark the image with the features of the mask. Marking uses the simple blob detector and the
    given colour.
    '''
    _, contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    retrieved_image = cv2.drawContours(image, contours, -1, colour, thickness=4)
    moments = [cv2.moments(contour) for contour in contours]
    retrieved_points = [[int(moment["m10"] / moment["m00"]), int(moment["m01"] / moment["m00"])]
        for moment in moments]
    return retrieved_image, retrieved_points


def markCorners(image):
    '''
    Identify the corners of the maze.
    '''
    cyan = maskColour(image, (89, 50, 0), (91, 255, 255)) # HSV colours
    magenta = maskColour(image, (149, 100, 150), (151, 255, 255)) # HSV colours
    cyan_features = markFeature(image, cyan, (255, 0, 255)) # RGB colours
    magenta_features = markFeature(image, magenta, (0, 255, 255)) # RGB colours
    retrieved_image = cyan_features[0] | magenta_features[0]
    return retrieved_image, cyan_features[1], magenta_features[1]


def gradient_angle(point1, point2):
    '''
    Helper function to calculate the angle of the gradient between two points. This only works for
    angles between (-90, 90)
    '''
    try:
        return np.rad2deg(np.arctan((point2[1] - point1[1]) / (point2[0] - point1[0]) % (2 * np.pi)))
    except ZeroDivisionError:
        return np.rad2deg(np.pi)


def closest_collinearity(point1, point2, point3):
    '''
    Helper function which returns a scalar for collinearity between 3 points. The smaller the number,
    the better the collinearity. The expressions used is derived from the gradient relationship
    between the 3 points.
    '''
    a = (point1[1] - point2[1]) * (point1[0] - point3[0])
    b = (point1[1] - point3[1]) * (point1[0] - point2[0])
    return abs(a - b)


def closest_distance(point1, point2):
    '''
    Helper function which returns a scalar for the closest distance between 2 points. The smaller the
    number, the closer the distance.
    '''
    return (point1[0] - point2[0])**2 + (point1[1] - point2[1])**2


def sortCorners(image, points):
    '''
    Sorts the MAGENTA markers so that the orientation of the perspective of the map will have the
    CYAN marker at the top-left corner. Assumes that blue walls are always running north-south.
    '''
    # Mask the image for the blue wall which we know will always be running north-south.
    blue = maskColour(image, (119, 50, 0), (121, 255, 255))

    # Get the contours of the blue wall.
    blue_features = markFeature(image, blue, None)

    # Create combinations of 3 points to test for collinearity where 2 of the points are corner
    # markers and the third is the centroid of the contour.
    temp_points = [[u, v, w] for u, v in it.combinations(points, 2) for w in blue_features[1]]

    # Get the set of points that is closest in collinearity.
    collinear_points = min(temp_points, key=lambda k: closest_collinearity(k[0], k[1], k[2]))
    
    # Delete the centroid point because it is no longer needed.
    del collinear_points[-1]

    # Get the corner point that is not part of the collinearity set
    temp_point = [p for p in points if p not in collinear_points]

    # Put the corner point not in collinearity set at beginning of list and sort the other two
    # corner points for correct clockwise alignment.
    return temp_point + [p for p in sorted(collinear_points, key=lambda q: closest_distance(temp_point[0], q))]


def focus(image, origin, points):
    '''
    Focus the perspective of the map. The corners of the map are at the corners of the image, the
    view is orthogonal with the ground, the width to height ratio is 9:5.
    '''
    if origin is None:
        raise ValueError('No given origin point.')
    if len(points) != 3:
        raise ValueError('Invalid number of markers: ', len(points), '.')
    width, height = 900, 500
    output_points = np.float32([[0, 0], [0, height-1], [width-1, height-1], [width-1, 0]])
    input_points = np.float32([origin + sortCorners(image, points)])
    transform_matrix = cv2.getPerspectiveTransform(input_points, output_points)
    retrieved_image = cv2.warpPerspective(image, transform_matrix, (width, height), cv2.INTER_LINEAR)
    return retrieved_image


def detectWalls(image):
    pass


def detectPose():
    pass


def generateMap():
    pass


In [ ]:
if __name__ == '__main__':
    '''
    Task 1
    '''
    image = readImage()
    plt.figure(figsize=(13,13))
    plt.imshow(image)

In [ ]:
if __name__ == '__main__':
    '''
    Task 2
    '''
    image = readImage()
    image, _, _ = markCorners(image)
    plt.figure(figsize=(13,13))
    plt.imshow(image)

In [ ]:
if __name__ == '__main__':
    '''
    Task 3
    '''
    image = readImage()
    image, origin, points = markCorners(image)
    image = focus(image, origin, points)
    plt.figure(figsize=(13,13))
    plt.imshow(image)

In [ ]:
if __name__ == '__main__':
    '''
    Task 4
    '''
    image = readImage()
    image, origin, points = markCorners(image)
    image = focus(image, origin, points)
    image = detectWalls(image)
    plt.figure(figsize=(13,13))
    plt.imshow(image)